In [1]:
import numpy as np
import time
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
class TrumpFaceDataset(Dataset):
    def __init__(self, path, transform=None):
        self.X = np.load("{}/X.npy".format(path))
        self.Y = np.load("{}/Y.npy".format(path))
        self.X = np.asarray(self.X, dtype=np.float)
        self.Y = np.asarray(self.Y, dtype=np.int)
        self.X = self.X[:160]
        self.Y = self.Y[:160]
        print("X: {}, Y: {}".format(self.X.shape, self.Y.shape))
        print("OTHER: {}, TRUMP: {}".format(np.sum(self.Y == 0), np.sum(self.Y == 1)))
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):       
        if self.transform:
            return self.transform(self.X[idx]), self.Y[idx]
        else:
            return self.X[idx], self.Y[idx]

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 61 * 61, 512)
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = x.float()
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 61 * 61)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
batchsize = 32
lr = 0.001
momentum = 0.9
n_epoch = 10000
divisor = 10

net = Net().to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

datapath = "/home/henning/git_repos/htwg_mldl/Versuche/data/TrumpFace"
classes = ("OTHER", "TRUMP")
trumpface_dataset = TrumpFaceDataset(datapath, transform=transform)
dataset_size = len(trumpface_dataset)

trainloader = DataLoader(trumpface_dataset, 
                         shuffle=True, 
                         batch_size=batchsize, 
                         num_workers=4)

start = time.time()
for epoch in range(n_epoch):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        outputs = net(inputs).to('cuda')
        loss = criterion(outputs, labels).to('cuda')
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if (epoch % divisor) == 0:
        class_correct = list(0. for i in range(len(classes)))
        class_total = list(0. for i in range(len(classes)))
        with torch.no_grad():
            for data in trainloader:
                images, true_labels = data
                images = images.to('cuda')
                true_labels = true_labels.to('cuda')
                outputs = net(images).to('cuda')
                _, predicted = torch.max(outputs, 1)
                c = (predicted == true_labels).squeeze()

                for i in range(batchsize):
                    label = true_labels[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1
                
        elapsed = (time.time() - start)
        e_h = int(elapsed // 3600)
        e_min = int((elapsed - (e_h * 3600)) // 60)
        e_sec = int(elapsed - (e_h * 3600) - (e_min * 60))
        t = "elapsed time: {}:{:02d}:{:02d}".format(e_h, e_min, e_sec)
        
        elapsed /= epoch + 1
        e_h = int(elapsed // 3600)
        e_min = int((elapsed - (e_h * 3600)) // 60)
        e_sec = int(elapsed - (e_h * 3600) - (e_min * 60))
        te = "time/epoch: {}:{:02d}:{:02d}".format(e_h, e_min, e_sec)
        
        print("epoch: {:d}, loss: {:.3f}, {}, {}".format(epoch + 1, running_loss / divisor, t, te))
        
        for i in range(len(classes)):
            print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))
        print()
        
        running_loss = 0.0
print('Finished Training')

X: (160, 256, 256, 3), Y: (160,)
OTHER: 61, TRUMP: 99
epoch: 1, loss: 336.808, elapsed time: 0:00:01, time/epoch: 0:00:01
Accuracy of OTHER :  0 %
Accuracy of TRUMP : 100 %

epoch: 11, loss: 0.028, elapsed time: 0:00:12, time/epoch: 0:00:01
Accuracy of OTHER : 96 %
Accuracy of TRUMP : 98 %

epoch: 21, loss: 0.004, elapsed time: 0:00:23, time/epoch: 0:00:01
Accuracy of OTHER : 100 %
Accuracy of TRUMP : 100 %

epoch: 31, loss: 0.038, elapsed time: 0:00:34, time/epoch: 0:00:01
Accuracy of OTHER : 100 %
Accuracy of TRUMP : 96 %

epoch: 41, loss: 0.244, elapsed time: 0:00:45, time/epoch: 0:00:01
Accuracy of OTHER : 93 %
Accuracy of TRUMP : 94 %

epoch: 51, loss: 0.019, elapsed time: 0:00:56, time/epoch: 0:00:01
Accuracy of OTHER : 100 %
Accuracy of TRUMP : 96 %

epoch: 61, loss: 0.006, elapsed time: 0:01:07, time/epoch: 0:00:01
Accuracy of OTHER : 100 %
Accuracy of TRUMP : 98 %

epoch: 71, loss: 0.003, elapsed time: 0:01:18, time/epoch: 0:00:01
Accuracy of OTHER : 100 %
Accuracy of TRUMP : 

KeyboardInterrupt: 